In [1]:
%connect_info

{
  "shell_port": 57328,
  "iopub_port": 57329,
  "stdin_port": 57330,
  "control_port": 57332,
  "hb_port": 57331,
  "ip": "127.0.0.1",
  "key": "78985efa-7fbd72bc5e451fce4ef811a7",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-5ab1cbd7-8ec4-4779-b8e4-4392728a67fd.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [2]:
%matplotlib inline

## Possible models

`bert-base-multilingual-cased`: (New, recommended) 12-layer, 768-hidden, 12-heads, 110M parameters. Trained on cased text in the top 104 languages with the largest Wikipedias

`xlm-mlm-100-1280`: 16-layer, 1280-hidden, 16-heads XLM model trained with MLM (Masked Language Modeling) on 100 languages.

`distilbert-base-multilingual-cased`: 6-layer, 768-hidden, 12-heads, 134M parameters The multilingual DistilBERT model distilled from the Multilingual BERT model bert-base-multilingual-cased checkpoint.

In [3]:
import sys
sys.argv = sys.argv[:1]

In [4]:
import os
import pickle
import numpy as np
import tensorflow as tf

from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from evaluation import  evaluate, MetricsReporterCallback
from utils import build_model_name, convert_flags_to_dict, define_cnn_flags
from preprocess import load_and_tokenize_dataset

from transformers import AutoTokenizer, TFAutoModel

from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [5]:
SEED = 42
BASE_DIR = os.path.expanduser("~")     # this will point to the user's home
TRAIN_DIR = "ray_results"

FLAGS = define_cnn_flags(tf.compat.v1.flags, BASE_DIR, TRAIN_DIR)

In [6]:
model_type = 'distilbert-base-multilingual-cased'

In [7]:
# tokenizer = AutoTokenizer.from_pretrained(model_type)
# transformer = TFAutoModel.from_pretrained(model_type)

In [8]:
FLAGS.conv_layers = [int(i) for i in FLAGS.conv_layers]
FLAGS.dense_layers = [int(i) for i in FLAGS.dense_layers]

_config = convert_flags_to_dict(FLAGS)
_config["codes"] = (["DE', 'GA', 'HI', 'PT', 'ZH"]
                    if FLAGS.language_code is 'all' else [FLAGS.language_code])

cwd = os.getcwd()

RESULTS = {}

In [9]:

def train_model(config):

    model_name = build_model_name(config)

    with open('{}/data/{}.tokenized.pkl'.format(cwd, config["bert_type"]),
              'rb') as f:
        data = pickle.load(f)

    x_train, y_train, x_dev, y_dev = [], [], [], []
    for code in config["codes"]:
        x_train += data[code]["x_train"]
        y_train += data[code]["y_train"]

        x_dev += data[code]["x_dev"]
        y_dev += data[code]["y_dev"]

    del data
    
    x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train)
    max_len = x_train.shape[1]
    y_train = tf.keras.preprocessing.sequence.pad_sequences(y_train, maxlen=max_len)


    x_train, x_val, y_train, y_val = train_test_split(x_train,
                                                      y_train,
                                                      test_size=0.15,
                                                      random_state=SEED)

    print(x_train.shape, x_train.shape, y_train.shape)
    x_train = [x_train, (x_train > 0).astype(int)]
    x_val = [x_val, (x_val > 0).astype(int)]

    x_dev = tf.keras.preprocessing.sequence.pad_sequences(x_dev, maxlen=max_len)
    x_dev = [x_dev, (x_dev > 0).astype(int)]

    seq_lens = [len(seq) for seq in y_dev]
    y_dev = tf.keras.preprocessing.sequence.pad_sequences(y_dev, maxlen=max_len)
    print(x_dev[0].shape, x_dev[1].shape, y_dev.shape)

   

    conv_config = ([config["conv_size"]] * config["nconv"]
                   if config["nconv"] > 0 and config["conv_size"] > 0
                   else config["conv_layers"])

    dense_config = ([config["dense_size"]] * config["ndense"]
                    if config["ndense"] > 0 and config["dense_size"] > 0
                    else config["dense_layers"])


    input_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32)

    # embedding
    transformer = TFAutoModel.from_pretrained(config["bert_type"])
    out = transformer(
        input_ids, attention_mask=attention_mask, training=False)[0]

    for i, layer_size in enumerate(conv_config):
        if i == 0:
            out = tf.keras.layers.Conv1D(
                layer_size,
                config["strides"],
                padding='same',
                activation=config["conv_activation"],
                strides=1,
                input_shape=(None, x_train[0].shape[0], x_train[0].shape[1]))(out)
        else:
            out = tf.keras.layers.Conv1D(
                layer_size,
                config["strides"],
                padding='same',
                activation=config["conv_activation"],
                strides=1)(out)
        out = tf.keras.layers.Dropout(config["conv_dropout"])(out)

    # Dense layers
    for i, layer_size in enumerate(dense_config):
        out = tf.keras.layers.Dense(
            layer_size, activation=config["dense_activation"])(out)
        out = tf.keras.layers.Dropout(config["dense_dropout"])(out)

    if config["output_size"] == 1:
        out = tf.keras.layers.Dense(1, activation='sigmoid')(out)
    else:
        out = tf.keras.layers.Dense(2, activation=config["output_activation"])(out)

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=out)

    if config["optimizer"] == 'adam':
        optimizer = tf.keras.optimizers.Adam
    elif config["optimizer"] == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop
    else:
        optimizer = tf.keras.optimizers.SGD

    # compiling model
    model.compile(loss=config["loss_function"],
                  optimizer=optimizer(learning_rate=config["learning_rate"],
                                      clipnorm=config["clipnorm"]),
                  metrics=["accuracy"])

    print(model.summary())


    # do this check again vecause we need y_train to be 1-D for class weights
    if config["output_size"] > 1:
        y_train = tf.keras.utils.to_categorical(y_train)
        y_val = tf.keras.utils.to_categorical(y_val)

    checkpoint = tf.keras.callbacks.ModelCheckpoint(config["train_dir"] +
                                                    model_name,
                                                    save_best_only=True)
    callbacks = [checkpoint]

    if config["tune"]:
        callbacks.append(
            MetricsReporterCallback(custom_validation_data=(x_val, y_val)))

    if config["early_stop_patience"] > 0:
        early_stop = tf.keras.callbacks.EarlyStopping(
            monitor='loss',
            min_delta=config["early_stop_delta"],
            patience=config["early_stop_patience"])
        callbacks.append(early_stop)

    if config["log_tensorboard"]:
        tensorboard = tf.keras.callbacks.TensorBoard(
            log_dir=config["train_dir"] + '/logs')
        callbacks.append(tensorboard)

    def lr_scheduler(epoch, lr):     # pylint: disable=C0103
        lr_decay = config["lr_decay"]**max(epoch - config["start_decay"], 0.0)
        return lr * lr_decay

    if config["start_decay"] > 0:
        lrate = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)
        callbacks.append(lrate)

    print(model.summary())

    print('Train...')
    model.fit(x_train,
              y_train,
              batch_size=config["batch_size"],
              epochs=config["max_epochs"],
              callbacks=callbacks,
              verbose=2,
              validation_data=(x_val, y_val))

    # #####
    # Evaluation time
    #
    _results, y_pred = evaluate(
        model,
        test_data=(x_dev[0], y_dev),
        perword=True,
        seq_lens=seq_lens,
        output_dict=True)


    logs = {
        "dev_label0_support" :_results["0"]["support"],
        "dev_precision" :_results["1"]["precision"],
        "dev_recall" :_results["1"]["recall"],
        "dev_f1_score" :_results["1"]["f1-score"],
        "dev_label1_support" :_results["1"]["support"],
        "dev_macro_precision" :_results["macro avg"]["precision"],
        "dev_macro_recall" :_results["macro avg"]["recall"],
        "dev_macro_f1_score" :_results["macro avg"]["f1-score"],
        "dev_weighted_precision" :_results["weighted avg"]["precision"],
        "dev_weighted_recall" :_results["weighted avg"]["recall"],
        "dev_weighted_f1_score" :_results["weighted avg"]["f1-score"]}

    trial_id = ray.tune.track.trial_id()

    RESULTS[str(trial_id)] = logs

    output_count = -1
    y_pred = y_pred.reshape(-1,).tolist()
    for code in config["codes"]:
        with open('{}/data/{}/dev.cupt'.format(cwd, code), 'r') as dev:
            with open('{}/data/{}/dev.cupt'.format(cwd, code)) as test:
                for line in dev:
                    if not line.startswith('#') and line is not '\n':
                        test.write(line.replace('*', y_pred[output_count]))
                        output_count += 1
                    else:
                        test.write(line)


In [10]:
_config['max_epochs'] = 1
_config["codes"] = ['GA']

In [ ]:
train_model(_config)


Model name nn_classifier.distilbert-base-multilingual-cased.all.1epochs.10-0.0010eStop.2-100-sigmoidlayers1strides.-0.100dropout2-sigmoid.output.0.5.thresh.weighted.binary_crossentropyLoss.32batch.sgd.0.0001lr.0.8696-0decay.5.00norm.ckpt

(218, 336) (218, 336) (218, 336)
(322, 336) (322, 336) (322, 336)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 336)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 336)]        0                                            
__________________________________________________________________________________________________
tf_d